In [16]:
using Statistics: mean  # standard library
function loss_and_accuracy(model, data)

    (x,y) = only(loader(data; batchsize=length(data)))
    ŷ = model(x)

    loss = Flux.logitcrossentropy(ŷ, y)  # did not include softmax in the model
    acc = round(100 * mean(Flux.onecold(ŷ) .== Flux.onecold(y)); digits=2) # maxa z kolekcji
    (; loss, acc, split=data.split)  # return a NamedTuple
end

loss_and_accuracy (generic function with 1 method)

In [18]:
# Opracowane na podstawie https://github.com/FluxML/model-zoo/blob/3e91af32ebfad628b616618b11bfff2f9f519bec/vision/conv_mnist/conv_mnist.jl
using MLDatasets, Flux
train_data = MLDatasets.MNIST(split=:train)
test_data  = MLDatasets.MNIST(split=:test)

function loader(data; batchsize::Int=1)
    x4dim = reshape(data.features, 28, 28, 1, :) # insert trivial channel dim
    yhot  = Flux.onehotbatch(data.targets, 0:9)  # make a 10×60000 OneHotMatrix
    Flux.DataLoader((x4dim, yhot); batchsize, shuffle=true)
end

net = Chain(
    Conv((3, 3), 1 => 6,  relu, bias=false),
    MaxPool((2, 2)),
    Flux.flatten,
    Dense(13*13*6 => 84, relu, bias=false), 
    Dense(84 => 10, identity, bias = false)
)


Chain(
  Conv((3, 3), 1 => 6, relu, bias=false),  # 54 parameters
  MaxPool((2, 2)),
  Flux.flatten,
  Dense(1014 => 84, relu; bias=false),  # 85_176 parameters
  Dense(84 => 10; bias=false),          # 840 parameters
)                   # Total: 3 arrays, 86_070 parameters, 336.820 KiB.

In [26]:
train_log = []
settings = (;
    eta = 1e-2,
    epochs = 3,
    batchsize = 100,
)

opt_state = Flux.setup(Descent(settings.eta), net);

In [27]:
for epoch in 1:settings.epochs
    @time for (x,y) in loader(train_data, batchsize=settings.batchsize)
        grads = Flux.gradient(model -> Flux.logitcrossentropy(model(x), y), net)
        Flux.update!(opt_state, net, grads[1])
    end
    
    loss, acc, _ = loss_and_accuracy(net, train_data)
    test_loss, test_acc, _ = loss_and_accuracy(net, test_data)
    @info epoch acc test_acc
    nt = (; epoch, loss, acc, test_loss, test_acc) 
    push!(train_log, nt)
end

  5.437795 seconds (1.10 M allocations: 4.842 GiB, 11.63% gc time, 8.16% compilation time)


┌ Info: 1
│   acc = 88.89
│   test_acc = 89.72
└ @ Main d:\Projects\CNNLibrary\reference_solution.ipynb:9


  5.354702 seconds (264.29 k allocations: 4.787 GiB, 12.62% gc time)


┌ Info: 2
│   acc = 91.23
│   test_acc = 91.82
└ @ Main d:\Projects\CNNLibrary\reference_solution.ipynb:9


  5.261762 seconds (264.29 k allocations: 4.787 GiB, 11.61% gc time)


┌ Info: 3
│   acc = 92.56
│   test_acc = 93.0
└ @ Main d:\Projects\CNNLibrary\reference_solution.ipynb:9
